<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/Forex_Price_Prediction_with_LSTM_and_Optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Import necessary libraries

!pip install pandas_ta
!pip install yfinance
!pip install optuna

import optuna
from sklearn.model_selection import TimeSeriesSplit

import datetime
import yfinance as yf
import numpy as np
import pandas as pd
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas_ta as ta

# Gather Data
symbol = "AUDUSD=X"
start_date = "2010-01-01"
end_date = datetime.datetime.now().strftime("%Y-%m-%d")
df = yf.download(symbol, start=start_date, end=end_date, interval="1d")

# Calculate technical indicators
df["RSI"] = ta.rsi(df["Close"])
df[["MACD_12_26", "MACD_12_26_Signal", "MACD_12_26_Hist"]] = ta.macd(df["Close"])
df["ATR"] = ta.atr(df["High"], df["Low"], df["Close"])

# Drop NaN values
df.dropna(inplace=True)

# Preprocess Data
df_scaler = MinMaxScaler()
input_features = ["Close", "RSI", "MACD_12_26", "MACD_12_26_Signal", "MACD_12_26_Hist", "ATR"]
df[input_features] = df_scaler.fit_transform(df[input_features])

close_scaler = MinMaxScaler()
df['Close'] = close_scaler.fit_transform(df[['Close']])

# Reset index
df.reset_index(drop=True, inplace=True)

# Split Data
train_size = int(len(df) * 0.8)
train_data = df[:train_size]
test_data = df[train_size:]

# Create sequences
def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length - 1):
        sequence = data[i : i + seq_length].values
        label = data["Close"].iloc[i + seq_length]
        sequences.append(sequence)
        labels.append(label)
    return np.array(sequences), np.array(labels)

seq_length = 60
train_x, train_y = create_sequences(train_data[input_features], seq_length)
test_x, test_y = create_sequences(test_data[input_features], seq_length)

# Define the objective function for Optuna
def objective(trial):

    # Hyperparameter search space
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])
    lstm_units = trial.suggest_categorical("lstm_units", [30, 50, 80])
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5, step=0.1)

    # Model definition
    model = Sequential()
    model.add(LSTM(units=lstm_units, return_sequences=True, input_shape=(train_x.shape[1], train_x.shape[2])))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=lstm_units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1))
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mean_squared_error")

    # TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=5)

    for train_index, val_index in tscv.split(train_x):
        train_x_cv, val_x_cv = train_x[train_index], train_x[val_index]
        train_y_cv, val_y_cv = train_y[train_index], train_y[val_index]

        model.fit(train_x_cv, train_y_cv, epochs=10, batch_size=batch_size, validation_data=(val_x_cv, val_y_cv), verbose=0)

    val_loss = model.evaluate(val_x_cv, val_y_cv, verbose=0)
    return val_loss

# Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

print("Best hyperparameters: ", study.best_params)

# Training the model with best hyperparameters on full data
best_params = study.best_params
model = Sequential()
model.add(LSTM(units=best_params["lstm_units"], return_sequences=True, input_shape=(train_x.shape[1], train_x.shape[2])))
model.add(Dropout(best_params["dropout_rate"]))
model.add(LSTM(units=best_params["lstm_units"]))
model.add(Dropout(best_params["dropout_rate"]))
model.add(Dense(units=1))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=best_params["learning_rate"]), loss="mean_squared_error")
model.fit(train_x, train_y, epochs=10, batch_size=best_params["batch_size"])

# ... (Rest of the code remains the same, for predictions, evaluations, etc.)


# Train Model
model.fit(train_x, train_y, epochs=10, batch_size=32)

# Make Predictions
predictions = model.predict(test_x)
predictions = close_scaler.inverse_transform(predictions)

# Evaluate Model
mse = mean_squared_error(predictions, close_scaler.inverse_transform(test_y.reshape(-1, 1)))
mae = mean_absolute_error(predictions, close_scaler.inverse_transform(test_y.reshape(-1, 1)))
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)


print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)

# Save Model
model.save("forex_trading_bot_model.keras")

# Make a prediction for the next day
next_day_sequence = np.expand_dims(test_x[-1], axis=0)
next_day_prediction = model.predict(next_day_sequence)
next_day_price = close_scaler.inverse_transform(next_day_prediction)[0][0]

# Print the predicted price for the next day
print("Predicted price for the next day:", next_day_price)

# Determine the action based on the predicted price
if next_day_price > df["Close"].iloc[-1]:
    print("Action: Buy")
elif next_day_price < df["Close"].iloc[-1]:
    print("Action: Sell")
else:
    print("Action: Hold")

# Create a baseline model that predicts the last known price as the next day's price
baseline_predictions = np.roll(test_x[:,-1,0], -1)  # Shift the last known prices one step forward
baseline_mse = mean_squared_error(test_y[:-1], baseline_predictions[:-1])
baseline_mae = mean_absolute_error(test_y[:-1], baseline_predictions[:-1])

# Calculate the Mean Squared Error (MSE) and Mean Absolute Error (MAE) for the LSTM model
lstm_mse = mean_squared_error(test_y, predictions)
lstm_mae = mean_absolute_error(test_y, predictions)

# Print the comparison results
print("\nBaseline Model:")
print("MSE:", baseline_mse)
print("MAE:", baseline_mae)

print("\nLSTM Model:")
print("MSE:", lstm_mse)
print("MAE:", lstm_mae)

# Compare the LSTM model's performance with the baseline model's performance
if lstm_mse < baseline_mse and lstm_mae < baseline_mae:
    print("\nConclusion: The LSTM model's performance is better than the baseline model. Consider using LSTM predictions for trading forex.")
    if next_day_price > df["Close"].iloc[-1]:
        print("Action: Buy")
    elif next_day_price < df["Close"].iloc[-1]:
        print("Action: Sell")
    else:
        print("Action: Hold")
elif lstm_mse > baseline_mse and lstm_mae > baseline_mae:
    print("\nConclusion: The baseline model's performance is better than the LSTM model. Caution advised.")
else:
    print("\nConclusion: The LSTM model's performance is similar to the baseline model. Use additional analysis for decision-making.")

[*********************100%%**********************]  1 of 1 completed


[I 2023-08-28 20:47:38,826] A new study created in memory with name: no-name-28434c2c-ae42-469c-b38a-9a930837b825
[I 2023-08-28 20:49:26,885] Trial 0 finished with value: 0.002494402229785919 and parameters: {'learning_rate': 0.00012906015402951772, 'batch_size': 128, 'lstm_units': 50, 'dropout_rate': 0.2}. Best is trial 0 with value: 0.002494402229785919.
[I 2023-08-28 20:53:46,025] Trial 1 finished with value: 0.00029523370903916657 and parameters: {'learning_rate': 0.0005609535011247337, 'batch_size': 64, 'lstm_units': 80, 'dropout_rate': 0.30000000000000004}. Best is trial 1 with value: 0.00029523370903916657.
[I 2023-08-28 20:57:16,111] Trial 2 finished with value: 0.0003992709389422089 and parameters: {'learning_rate': 0.0001450620032159825, 'batch_size': 128, 'lstm_units': 80, 'dropout_rate': 0.5}. Best is trial 1 with value: 0.00029523370903916657.
[I 2023-08-28 20:59:14,617] Trial 3 finished with value: 0.022772466763854027 and parameters: {'learning_rate': 1.6185631358174995e

Best hyperparameters:  {'learning_rate': 0.008619943206886924, 'batch_size': 16, 'lstm_units': 50, 'dropout_rate': 0.1}
Epoch 1/10
173/173 [==============================] - 16s 68ms/step - loss: 0.0087
Epoch 2/10
173/173 [==============================] - 12s 68ms/step - loss: 0.0022
Epoch 3/10
173/173 [==============================] - 12s 71ms/step - loss: 0.0017
Epoch 4/10
173/173 [==============================] - 12s 68ms/step - loss: 0.0015
Epoch 5/10
173/173 [==============================] - 12s 71ms/step - loss: 0.0012
Epoch 6/10
173/173 [==============================] - 12s 68ms/step - loss: 9.0417e-04
Epoch 7/10
173/173 [==============================] - 11s 65ms/step - loss: 8.2522e-04
Epoch 8/10
173/173 [==============================] - 12s 72ms/step - loss: 7.1226e-04
Epoch 9/10
173/173 [==============================] - 12s 71ms/step - loss: 8.9683e-04
Epoch 10/10
173/173 [==============================] - 12s 71ms/step - loss: 6.4481e-04
Epoch 1/10
87/87 [===========

Advanced Forex Analysis and Prediction using LSTM and Optuna

This notebook employs a deep learning approach to forecast forex prices using Long Short-Term Memory (LSTM) networks, a type of recurrent neural network that's well-suited for sequence prediction problems. The notebook's functionality can be broken down as follows:

Data Collection:

The notebook fetches historical forex data for a given currency pair (e.g., AUDUSD) from Yahoo Finance, spanning from the beginning of 2010 to the present day.
Feature Engineering:

It calculates multiple technical indicators, such as the Relative Strength Index (RSI), Moving Average Convergence Divergence (MACD), and Average True Range (ATR), which serve as additional features for the model.
Data Preprocessing:

The forex data, along with the technical indicators, are scaled using a MinMaxScaler for better model performance.
It splits the data into training and testing sets.
Model Building and Training:

An LSTM-based neural network model is constructed. This type of model can capture the temporal relationships in time series data, making it particularly suited for forex price prediction.
The model is trained on the historical data.
Evaluation:

The trained model's predictions on test data are evaluated against actual prices using Mean Squared Error (MSE) and Mean Absolute Error (MAE) metrics.
A baseline model, which predicts the last known price as the next day's price, is also evaluated. This helps in gauging the LSTM model's performance relative to a simple prediction strategy.
Hyperparameter Tuning and Cross-Validation:

The notebook integrates the TimeSeriesSplit method to perform time series cross-validation, ensuring that the model is validated on different segments of the dataset.
Optuna, a hyperparameter optimization framework, is employed to search for optimal parameters like learning rate, batch size, LSTM units, etc., aiming to enhance the model's performance.
Action Recommendation:

Based on the LSTM model's prediction for the next day's price and the most recent known price, the notebook suggests a trading action: Buy, Sell, or Hold.
Conclusion:

The notebook concludes by comparing the LSTM model's performance against the baseline model, advising the user on the best model to consider for forex trading decisions.